In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

base_path = "../build/03_sine_waves"

values = pd.read_csv(f"{base_path}/combined.csv")

values['time'] = values.index * (1/44100)



values[-5000:].plot(x='time', figsize=(30, 5))


In [ ]:
fft = pd.read_csv(f"{base_path}/mags.csv", names=['Frequency Bins'])
fft['freq'] = fft.index * (44100)/(1024 * 1024)
fft.plot(x='freq', y='Frequency Bins', figsize=(10,5))

In [ ]:
# Create a Hann window
x = np.linspace(0, len(values.index), len(values.index))
y = 0.5 * (1 - np.cos(2.0 * np.pi * x / len(values.index)))

plt.plot(x, y)

In [ ]:
window = pd.read_csv(f"{base_path}/window.csv")
window.plot()

In [ ]:
fft = pd.read_csv(f"{base_path}/video_mags.csv", names=['Frequency Bins'])
fft['freq'] = (fft.index % 4096) * (48000)/(4096)
#fft[:2048].plot(x='freq', y='Frequency Bins', figsize=(10,5))
num_plots = int(len(fft) / 4096 + 1)
fig, axs = plt.subplots(num_plots)

for i in range(num_plots):
    fft[i * 4096:(i+1)*4096-1].plot(x='freq', y='Frequency Bins', ax=axs[i], figsize=(10,20))